In [ ]:
#Mounting Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Linux library installations
!sudo apt-get update
!sudo apt-get -y install sox ffmpeg libfreeimage-dev imagemagick

In [ ]:
#pip installations
!pip install google-cloud-speech google-cloud-storage srt readchar 
!sudo pip3 install imageio==2.4.1

In [ ]:
#Installations on local machine
#!pip install matplotlib readchar moviepy

In [ ]:
#python imports
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import re
import readchar
import srt
import sys
import time

from base64 import b64encode
from google.cloud import speech, storage
from moviepy.editor import AudioFileClip, VideoFileClip, ImageClip, TextClip, CompositeVideoClip, concatenate_videoclips
from moviepy.video.fx.all import crop
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1 import ImageGrid
from skimage.filters import gaussian

from IPython.display import clear_output, HTML
from PIL import Image

In [ ]:
#Defining invariants
audio_upload_bucket_url = 'gs://sasha-clips-in/'
root_dir = '/content/drive/MyDrive/sasha_clips/' #google drive
#root_dir = '/Users/rohitsrao/My Drive/sasha_clips/' #mac
#'root_dir = '/home/rohitsrao/videos/sasha_clips/' chromebook

subfolders = [
    'landscape_logo',
    'landscape_logo_subtitles',
    'landscape_logo_subtitles_outro',
    'portrait_logo',
    'portrait_logo_subtitles',
    'portrait_logo_subtitles_outro',
    'square_logo',
    'square_logo_subtitles',
    'square_logo_subtitles_outro',
    'random_frames_camera_position_check',
    'logo_file_check',
    'subtitles', 
    'temp', 
    'temp/long_video_clip_processor'
]

mp4 = ".mp4"
png = ".png"
wav = ".wav"

clip_dims = {
    'landscape' : {
        'width': 1920,
        'height': 1080,
    },
    'portrait' : {
        'width': 608,
        'height': 1080,
    },
    'square' : {
        'width': 1080,
        'height': 1080,
    }
}

In [ ]:
logo_props = {
    'landscape' : {
        'left': {
            'symbol_position': (-35, -40),
            'symbol_resize_factor': 0.25,
            'text_position': (125, 52),
            'text_fontsize': 25,
        },
        'right': {
            'symbol_position': (1490, -40),
            'symbol_resize_factor': 0.25,
            'text_position': (1650, 52),
            'text_fontsize': 25,
        }
    },
    'portrait' : {
        'left': {
            #'symbol_position': (-30, 40), #original
            #'symbol_position': (-10, 80), #slight offset from left border
            'symbol_position': (0, 80), #slight offset from left border
            'symbol_resize_factor': 0.15,
            #'text_position': (65, 100), #original
            #'text_position': (85, 140), #slight offset from left border
            'text_position': (95, 140), #slight offset from left border
            'text_fontsize': 10,
        },
        'right': {
            #'symbol_position': (375, 40), #original
            'symbol_position': (355, 80), #shifted down
            'symbol_resize_factor': 0.15,
            #'text_position': (470, 100), #original
            'text_position': (450, 140),
            'text_fontsize': 10,
        }
    },
    'square' : {
        'left': {
            'symbol_position': (-30, -30),
            'symbol_resize_factor': 0.20,
            'text_position': (95, 47),
            'text_fontsize': 15,
        },
        'right': {
            'symbol_position': (775, -30),
            'symbol_resize_factor': 0.20,
            'text_position': (900, 47),
            'text_fontsize': 15,
        }
    },
}


logo_colours = {
    1: '#252d1d',
    2: '#b7bd9d',
    3: '#ffffff',
    4: '#e4dcd5',
    5: '#c1b3a5',
    6: '#403332',
}

In [ ]:
#post_animation_clip properties
post_clip_anim_props = {
    'landscape': {
        'gi_1': {
            'fontsize': 85,
        },
        'gi_2': {
            'fontsize': 85,
        },
        'with_sc': {
            'fontsize': 30,
            'pos_x': 0.585,
            'pos_y': 0.55
        },
        'watch_fv': {
            'fontsize': 40,
        },
        'at_sc': {
            'fontsize': 40
        },
        'final_logo_4': {
            'resize_factor': 0.65
        },
        'final_logo_5': {
            'resize_factor': 0.65
        }
    },
    'portrait': {
        'gi_1': {
            'fontsize': 35,
        },
        'gi_2': {
            'fontsize': 35,
        },
        'with_sc': {
            'fontsize': 15,
            'pos_x': 0.645,
            'pos_y': 0.52
        },
        'watch_fv': {
            'fontsize': 20,
        },
        'at_sc': {
            'fontsize': 20
        },
        'final_logo_4': {
            'resize_factor': 0.35
        },
        'final_logo_5': {
            'resize_factor': 0.35
        }
    },
    'square': {
        'gi_1': {
            'fontsize': 60,
        },
        'gi_2': {
            'fontsize': 60,
        },
        'with_sc': {
            'fontsize': 30,
            'pos_x': 0.54,
            'pos_y': 0.54
        },
        'watch_fv': {
            'fontsize': 40,
        },
        'at_sc': {
            'fontsize': 40
        },
        'final_logo_4': {
            'resize_factor': 0.5
        },
        'final_logo_5': {
            'resize_factor': 0.5
        }
    },
}

In [ ]:
#Environment Variables
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = root_dir + 'subclip-generator-bot-credentials.json'

In [ ]:
#Variables
current_video_dir = 'gi_e03/'
num_frame_checks = 12

In [ ]:
#Derived Variables
audio_dir = root_dir + 'audio/'
input_json_file = 'input_file.json'
input_json_file_location = root_dir + current_video_dir + input_json_file
logo_symbol_dir = root_dir + 'logos/logo_symbol_only/'
logo_name_dir = root_dir + 'logos/logo_name/'
original_video_file = root_dir + current_video_dir + current_video_dir[0:-1] + '.mp4' 
subclip_timestamps_file = root_dir + current_video_dir + 'subclip_timestamps.txt'
subtitles_dir = root_dir + current_video_dir + 'subtitles/'
temp_dir = root_dir + current_video_dir + 'temp/'
long_video_clip_processor_dir = temp_dir + 'long_video_clip_processor/'

In [ ]:
#cloud storage initializations
storage_client = storage.Client.from_service_account_json(root_dir + 'subclip-generator-bot-credentials.json')
bucket = storage_client.get_bucket('sasha-clips-in')

In [ ]:
#Setting path to ImageMagick Binary on MacOS
#from moviepy.config import change_settings
#change_settings({"IMAGEMAGICK_BINARY": "/opt/homebrew/Cellar/imagemagick/7.1.0-37_1/bin/convert"})

In [ ]:
#initialize ImageMagickPolicy so that MoviePy can detect ImageMagick
#so one line needs to be commented in policy.xml file located at /etc/ImageMagick-6/policy.xml
#line to be commented is 71 - <policy domain="path" rights="none" pattern="@*" />
#the file with line 71 commented is stored in root_dir
corrected_file_location = root_dir + 'policy.xml'
location_to_copy = '/etc/ImageMagick-6/policy.xml'
!sudo cp $corrected_file_location $location_to_copy

In [ ]:
#Copy fonts to system font dir
required_fonts = root_dir + 'fonts/*'
system_fonts_dir = '/usr/local/share/fonts'
!sudo cp -r $required_fonts $system_fonts_dir

In [ ]:
#create non-existing folders
for subfolder in subfolders:
    if not os.path.exists(root_dir + current_video_dir + subfolder):
        os.makedirs(root_dir + current_video_dir + subfolder)

In [ ]:
#Adding post clip animation
def add_post_clip_ani(input_params):

    #Extracting input parameters
    main_clip_end_time = input_params['end_time']
    title = input_params['title']
    mode = input_params['mode']
    crop_x_start = input_params['crop_x_start']
    
    #Defining how long we want the post clip animation to be
    #This value comes from correctly cutting the music at the 
    #beginning of the original file.
    post_clip_duration = 9.51 #seconds
    
    #Computing time steps for the complete clip
    post_clip_start = main_clip_end_time
    post_clip_end = timestamp_gen(post_clip_start, post_clip_duration)
    
    #Extracting the subclip
    post_clip = VideoFileClip(original_video_file).subclip(post_clip_start, post_clip_end)
    
    #Cropping
    post_clip_crop = crop_clip(post_clip, mode, crop_x_start)
    
    #Muting audio in post clip
    post_clip_mute = post_clip.volumex(0)
    
    #Defining the subclip durations of the post clip
    #We have 4 subclips based on the "eh" sound in the music
    
    #Defining the timestamps in a dictionary
    pc_t = {}
    
    pc_t['1_start'] = (0, 0, 0.0)
    pc_t['1_end'] = (0, 0, 1.111)
    pc_t['2_start'] = pc_t['1_end']
    pc_t['2_end'] = (0, 0, 3.544)
    pc_t['3_start'] = pc_t['2_end']
    pc_t['3_end'] = (0, 0, 6.040)
    pc_t['4_start'] = pc_t['3_end']
    pc_t['4_end'] = (0, 0, 8.47)
    pc_t['5_start'] = pc_t['4_end']
    pc_t['5_end'] = (0, 0, post_clip.duration)
    
    #Extracting the subclips corresponding to the subclip stamps
    #also applying blur after extraction
    #storing in a dictionary
    pc = {}
    for i in range(1, 6):
        pc[i] = post_clip_crop.subclip(pc_t[str(i)+'_start'], pc_t[str(i)+'_end']) 
        pc[i] = pc[i].fl_image(blur)

    #Extracting property values for current mode
    gi_1_fontsize = post_clip_anim_props[mode]['gi_1']['fontsize']
    gi_2_fontsize = post_clip_anim_props[mode]['gi_2']['fontsize']
    with_sc_fontsize = post_clip_anim_props[mode]['with_sc']['fontsize']
    with_sc_pos_x = post_clip_anim_props[mode]['with_sc']['pos_x']
    with_sc_pos_y = post_clip_anim_props[mode]['with_sc']['pos_y']
    watch_fv_fontsize = post_clip_anim_props[mode]['watch_fv']['fontsize']
    at_sc_fontsize = post_clip_anim_props[mode]['at_sc']['fontsize']
    final_logo_4_resize_factor = post_clip_anim_props[mode]['final_logo_4']['resize_factor']
    final_logo_5_resize_factor = post_clip_anim_props[mode]['final_logo_5']['resize_factor']
    
    #Defining text clips
    getting_intimate_1 = (TextClip('GETTING INTIMATE', 
                                   font='Montserrat-SemiBold', 
                                   fontsize=gi_1_fontsize, 
                                   color='white',
                                   align='center', 
                                   kerning=10)
                          .set_position('center', 'center')
                          .set_duration(pc[1].duration)
                          .crossfadein(0.3))
    
    getting_intimate_2 = (TextClip('GETTING INTIMATE', 
                                   font='Montserrat-SemiBold', 
                                   fontsize=gi_2_fontsize, 
                                   color='white',
                                   align='center', 
                                   kerning=10)
                          .set_position('center', 'center')
                          .set_duration(pc[2].duration)
                          .crossfadeout(0.3))
    
    with_sc = (TextClip('WITH SASHA COBRA', 
                        font='Montserrat-SemiBold', 
                        fontsize=with_sc_fontsize, 
                        color='white', 
                        align='center')
                        .set_position((with_sc_pos_x, with_sc_pos_y), relative=True)
                        .set_duration(pc[2].duration)
                        .crossfadein(0.3)
                        .crossfadeout(0.3))

    watch_fv = (TextClip('Watch full video series', 
                         font='Montserrat-SemiBold', 
                         fontsize=watch_fv_fontsize, 
                         color='white', 
                         align='center')
                        .set_position(('center', 'center'))
                        .set_duration(pc[3].duration)
                        .crossfadein(0.3)
                        .crossfadeout(0.3))
    
    at_sc = (TextClip('at sashacobra.com', 
                      font='Montserrat-SemiBold',
                      fontsize=at_sc_fontsize, 
                      color='white', 
                      align='center')
                      .set_position(('center', 0.52), relative=True)
                      .set_duration(pc[3].duration)
                      .crossfadein(0.3)
                      .crossfadeout(0.3))
    
    #Defining final logo image clip
    final_logo_4 = (ImageClip(logo_name_dir + 'logo_name_21.png')
                    .set_duration(pc[4].duration)
                    .set_pos(("center", "center"))
                    .resize(final_logo_4_resize_factor)
                    .crossfadein(0.3))
    
    final_logo_5 = (ImageClip(logo_name_dir + 'logo_name_21.png')
                    .set_duration(pc[5].duration)
                    .set_pos(("center", "center"))
                    .resize(final_logo_5_resize_factor)
                    .crossfadeout(pc[5].duration))
    
    #Generating composite video clips
    comp1 = CompositeVideoClip([pc[1], getting_intimate_1])
    comp2 = CompositeVideoClip([pc[2], getting_intimate_2, with_sc])
    comp3 = CompositeVideoClip([pc[3], watch_fv, at_sc])
    comp4 = CompositeVideoClip([pc[4], final_logo_4])
    comp5 = CompositeVideoClip([pc[5], final_logo_5])
    
    post_clip_with_text = concatenate_videoclips([comp1, comp2, comp3, comp4, comp5])
    
    #Setting post_clip audio

    #Importing audio file
    post_clip_audio = AudioFileClip(audio_dir + 'intro_audio.wav')
    
    #Setting audio
    post_clip_final = post_clip_with_text.set_audio(post_clip_audio)
    
    #Audio fade in and out effects
    post_clip_final = post_clip_final.audio_fadein(0.3)
    post_clip_final = post_clip_final.audio_fadeout(pc[5].duration)
    
    #post_clip_final.write_videofile(temp_dir + f'{title}_post_clip_animation', logger=None)
    
    return post_clip_final

In [ ]:
#Defining a function to blur a frame
def blur(image):
    
    """
    Returns a blurred image using gaussian filter
    with radius
    """
    
    return gaussian(image.astype(float), sigma=20)

In [ ]:
#breaks sentences from transcriptions
def break_sentences(subs, alternative):
    firstword = True
    charcount = 0
    idx = len(subs) + 1
    content = ""
    max_chars = 50

    for w in alternative.words:
        if firstword:
            # first word in sentence, record start time
            #start = w.start_time.ToTimedelta()
            start = w.start_time

        charcount += len(w.word)
        content += " " + w.word.strip()

        if ("." in w.word or "!" in w.word or "?" in w.word or
                charcount > max_chars or
                ("," in w.word and not firstword)):
            # break sentence at: . ! ? or line length exceeded
            # also break if , and not first word
            subs.append(srt.Subtitle(index=idx,
                                     start=start,
                                     #end=w.end_time.ToTimedelta(),
                                     end=w.end_time,
                                     content=srt.make_legal_content(content)))
            firstword = True
            idx += 1
            content = ""
            charcount = 0
        else:
            firstword = False
    return subs

In [ ]:
def spaces_in_title(title):
    if (len(title.split(' ')) > 0): return False
    else: return True

In [ ]:
def dot_or_comma_or_digit_error(time_string):
    pattern = r'\d{1},\d{2},\d{2}[.]\d{1,3}'
    if re.fullmatch(pattern, time_string) == None: return True
    else: return False

In [ ]:
#function to verify format of subclip timestamp
def verify_subclip_timestamp_format(clip):
    if (spaces_in_title(clip['title'])): print(f"Spaces in {clip['title']}")
    if (dot_or_comma_or_digit_error(clip['start'])): print(f"clip - {clip['title']} \n Dot or comma or digit error in start time - {clip['start']}")
    if (dot_or_comma_or_digit_error(clip['end'])): print(f"clip - {clip['title']} \n Dot or comma or digit error in end time - {clip['end']}")

In [ ]:
#function to parse subclip_timestamps_file
def subclip_timestamps_txt_parser():
    lines = []
    with open(subclip_timestamps_file, 'r') as f:
        lines = f.readlines()

    clips = []
    i = 0
    while (i+4 <= len(lines)+1):
        title = lines[i]
        print(f'processing {title}')
        start = lines[i+1]
        end = lines[i+2]
        d = {}
        d['title'] = title.strip()
        d['start'] = start.strip()
        d['end'] = end.strip()
        clips.append(d)
        i += 4
    return clips

In [ ]:
#function to check integrity of subtitle timestamps file
def check_subclip_timestamps_file():
    clips = subclip_timestamps_txt_parser()
    for clip in clips: verify_subclip_timestamp_format(clip)

#check_subclip_timestamps_file()

In [ ]:
#generate initial input json file
def generate_initial_input_json_file():
    fileData = {}
    clips = subclip_timestamps_txt_parser()
    for clip in clips:
        d = {}
        d['start_time'] = clip['start']
        d['end_time'] = clip['end']
        d['aspect_ratios'] = {
            "landscape": {
              "crop_x_start": 0,
              "logo_file": None,
              "logo_position": None
            },
            "portrait": {
              "crop_x_start": 0,
              "logo_file": None,
              "logo_position": None
            },
            "square": {
              "crop_x_start": 0,
              "logo_file": None,
              "logo_position": None
            }
        }
        fileData[clip['title']] = d
    
    with open(input_json_file_location, 'w') as f:
        json.dump(fileData, f, indent=2)
    
#generate_initial_input_json_file()

In [ ]:
#Loading input json file
with open(input_json_file_location) as f:
    subclip_data = json.load(f)

In [ ]:
#convert string time 'hh,mm,ss.millisec' to seconds
def convert_str_time_to_seconds(str_time):
    str_time = str_time.split('.')[0].split(',')
    str_time = [int(t) for t in str_time]
    return str_time[0]*60*60 + str_time[1]*60 + str_time[2]

In [ ]:
#function to crop the video
def crop_clip(clip, mode, crop_x_start=0):
    
    #Extracting the size
    w, h = clip.size

    width = clip_dims[mode]['width']
    
    #Cropping the clip 
    cropped_clip = crop(clip, x1=int(crop_x_start), width=int(width))
    
    return cropped_clip

In [ ]:
#Generate a subclip without crop
def generate_subclip_without_crop(video_file, start_time, end_time):
    return VideoFileClip(original_video_file).subclip(
        hmsms_to_tuple(start_time), 
        hmsms_to_tuple(end_time)
    )

In [ ]:
#generate subtitles from raw transcription response
def generate_subtitles_from_raw_transcription(response):
    subs = []

    for result in response.results:
        # First alternative is the most probable result
        subs = break_sentences(subs, result.alternatives[0])

    print("Transcribing finished")
    return subs

In [ ]:
#Convert input data string timestamp "hh,mm,ss.ms" into float tuple (hour, minute, seconds)
def hmsms_to_tuple(string_time):
    return tuple([float(element) for element in string_time.split(',')])

In [ ]:
#resize image
def img_resize(image_path, factor=None):
    img = Image.open(image_path).convert('RGB')
    if (factor != None): 
        img = img.resize(tuple([int(s/factor) for s in list(img.size)]))
    img = np.asarray(img)
    return img

In [ ]:
#convert audio file to single channel, 16KHz and 16bit
#runs shell command with sox
def prep_audio_for_speech2txt(title):
    original_audio_file = temp_dir + title + wav 
    encoded_audio_file = temp_dir + title + '_en' + wav
    !sox $original_audio_file -b 16 -r 16000 -c 1 $encoded_audio_file

In [ ]:
#Function to stitch subtitiles
#It generates the subclips for each subtitle section, 
#adds subtitles and creates a transition
def stitch_subtitles(subtitle, clip, mode):
    
    #Defining list for concatenation
    clips_for_concatenation = []
    
    #Defining sizes
    data = {
        'landscape': {
            'dims': (850, 110),
            'position': ('center', 960),
            'fontsize': 32
        },
        'portrait': {
            'dims': (450, 110),
            #'position': (30, 700),
            'position': (50, 800),
            'fontsize': 24
        },
        'square': {
            'dims': (800, 100),
            'position': ('center', 950),
            'fontsize': 28
        }
    }
    textclip_size = data[mode]['dims']
    subtitle_pos = data[mode]['position']
    subtitle_font_size = data[mode]['fontsize']
    
    #Looping through each subtitle section
    for i in subtitle.keys():
        
        #Extracting the clip that is in between two subtitles
        #and appending to the list for concatenation
        if i != len(subtitle.keys()):
            
            #Extracting the clip
            subtitle[i]['clip'] = clip.subclip(subtitle[i]['start'], subtitle[i]['end'])
        
            #Saving the duration
            subtitle[i]['duration'] = subtitle[i]['clip'].duration
        
            #Generating the text clip
            subtitle[i]['text_clip'] = TextClip(
                subtitle[i]['text'], 
                align='center',
                bg_color='#b6b4b2', 
                color='#071a13', 
                font='Ubuntu-Bold', 
                fontsize=subtitle_font_size, 
                method='caption', 
                size=textclip_size, 
            ).set_opacity(0.75)
        
            #Adding the text clip to the clip
            subtitle[i]['clip'] = CompositeVideoClip([subtitle[i]['clip'], subtitle[i]['text_clip']
                                                      .set_pos(subtitle_pos)]).set_duration(subtitle[i]['duration'])
        
            #Adding a transition clip for the bit in between 2 subtitles
            subtitle[i]['transition'] = clip.subclip(subtitle[i]['end'], subtitle[i+1]['start'])
            
            #Storing clips for concatenation
            clips_for_concatenation.append(subtitle[i]['clip'])
            clips_for_concatenation.append(subtitle[i]['transition'])
            
        #For the last clip, ensuring that it goes till the end
        else:
            
            #Extraction of clip
            subtitle[i]['clip'] = clip.subclip(subtitle[i]['start'], clip.duration)
            
            #Saving the duration
            subtitle[i]['duration'] = subtitle[i]['clip'].duration
        
            #Generating the text clip
            subtitle[i]['text_clip'] = TextClip(subtitle[i]['text'], font='Ubuntu-Bold', fontsize=subtitle_font_size, 
                                       color='#071a13', size=textclip_size, method='caption', bg_color='#b6b4b2', 
                                       align='center').set_opacity(0.75)
            
            #Adding the text clip the clip
            subtitle[i]['clip'] = subtitle[i]['clip'] = CompositeVideoClip([subtitle[i]['clip'], subtitle[i]['text_clip']
                                                      .set_pos(subtitle_pos)]).set_duration(subtitle[i]['duration'])
        
            #Storing clip for concatenation
            clips_for_concatenation.append(subtitle[i]['clip'])
            
    #Concatenating the clips
    main_clip = concatenate_videoclips(clips_for_concatenation, method='chain')
    
    return main_clip

In [ ]:
#convert subtitles from file into dict
def subtitle_file_to_dict(subtitle_file):
    
    #Extracting contents of srt file into a list
    with open(subtitle_file) as f:
        sub_content = f.readlines()
        
    #Defining a counter to keep track of the max number of 
    #subtitle sections
    max_subtitle_section = 0
    
    #Defining a dictionary to keep track of subtitle content
    #This is a dicionary that stores another dictionary
    subtitle = {}
    
    #Remove the \n at the end of each line
    sub_content = [line.rstrip() for line in sub_content]
        
    for i in range(len(sub_content)):
        
        #Check if string is number
        if sub_content[i].isdigit():
            
            #Create a temp dictionary
            temp = {}
            
            #Extract the timestamps
            timestamps = sub_content[i+1]
            
            #Split the timestamps by space
            timestamps = timestamps.split(' ')
            
            #Extracting the start and end time for the subtitle
            temp['start'] = timestamp_convert(timestamps[0])
            temp['end'] = timestamp_convert(timestamps[2])
            
            #Extract the subtitle text
            temp['text'] = sub_content[i+2]
            
            #Adding the temp dictionary to the subtitle dictionary
            subtitle[int(sub_content[i])] = temp
            
    return subtitle

In [ ]:
#Defining a function that can take a string of srt timestamp
#and convert it to format that MoviePy needs
def timestamp_convert(srt_time):
    
    '''
    Expects a string in the form 'hours:min:seconds,milliseconds'
    Returns (hours, minutes, seconds.milliseconds)
    '''
    
    #Splitting on the :
    srt_time = srt_time.split(':')
    
    #Extracting hour, minute and seconds
    hour = int(srt_time[0])
    minute = int(srt_time[1])
    seconds = float(srt_time[2].replace(',', '.'))
    
    #Converting to tuple
    moviepy_time = (hour, minute, seconds)
    
    return moviepy_time

In [ ]:
#Defining a function to generate timestamps for subclips
#Takes into account minute and seconds
def timestamp_gen(start, increment):
    """
    start - start time tuple (hours, min, seconds)
    increment - in seconds
    """
    
    #Setting stop tuple to be same as start
    stop = start
    
    #Converting to a list to increment as tuple is immutable
    temp_list = list(stop)
    
    #Adding length of post clip animation duration to the end of the main clip
    new_min = temp_list[2] + increment
    
    #Updating the list based on whether the incremented minute
    #is greater than or less than 60min as the hour has to be 
    #accounted for.
    if new_min < 60:
        temp_list[2] = new_min
    elif new_min > 60:
        temp_list[1] += 1
        temp_list[2] = new_min-60
        
    #Converting list back to tuple
    stop = tuple(temp_list)
    
    return stop

In [ ]:
#Transcribe long audio file from Cloud Storage using asynchronous speech recognition
def transcribe(storage_uri):
    sample_rate_hertz = 16000
    language_code = "en-US"
    audio_channel_count = 1
    encoding = "LINEAR16"

    print("Transcribing {} ...".format(storage_uri))
    client = speech.SpeechClient()

    # Encoding of audio data sent.
    operation = client.long_running_recognize(
        config=
        {
            "enable_word_time_offsets": True,
            "enable_automatic_punctuation": True,
            "sample_rate_hertz": sample_rate_hertz,
            "language_code": language_code,
            "audio_channel_count": audio_channel_count,
            "encoding": encoding,
        },
        audio={"uri": storage_uri},
    )
    response = operation.result()
    return response

In [ ]:
#upload processed audio file to storage bucket
def upload_audio_to_bucket(title):
    blob = bucket.blob(title)
    blob.upload_from_filename(temp_dir + title + '_en' + wav)

In [ ]:
#write subtitles to file
def write_srt(subs, title):
    srt_file = subtitles_dir + title + '.srt'
    language_code = "en-US"
    print("Writing {} subtitles to: {}".format(language_code, srt_file))
    f = open(srt_file, 'w')
    f.writelines(srt.compose(subs))
    f.close()
    return

In [ ]:
#subtitle file generation loop
#for title, data in subclip_data.items():
#    #if title != '04_life_gives_opportunities_to_heal': continue
#    #if int(title[0:2]) < 28: continue
#    print(f'generating subtitles for {title}')
#    subclip = generate_subclip_without_crop(original_video_file, data['start_time'], data['end_time'])
#    subclip.audio.write_audiofile(temp_dir + title + '.wav')
#    prep_audio_for_speech2txt(title)
#    upload_audio_to_bucket(title)
#    transcription_response = transcribe(audio_upload_bucket_url + title)
#    subtitles = generate_subtitles_from_raw_transcription(transcription_response)
#    write_srt(subtitles, title)
#    clear_output(wait=True)

In [ ]:
#generate clips with subtitles
#for title, data in subclip_data.items():
#    
#    if int(title[0:2]) < 30: continue
#    print(f'processing {title}')
#    subclip = generate_subclip_without_crop(original_video_file, data['start_time'], data['end_time'])
#    subtitle_file = subtitles_dir + title + '.srt'
#    
#    #Extract the data in subtiitle file
#    subtitle = subtitle_file_to_dict(subtitle_file)
#    
#    #Stitching the subtitles
#    clip_with_subtitles = stitch_subtitles(subtitle, subclip, 'square')
#
#    save_path = temp_dir + title + '.webm'
#    #clip_with_subtitles.write_videofile(save_path, codec='libvpx', ffmpeg_params=['-c:v', 'videotoolbox'])
#    clip_with_subtitles.write_videofile(save_path)
#
#    clear_output(wait=True)

In [ ]:
#re-assigning subtitle section numbers after editing and correcting subtitle files
#for file in sorted(os.listdir(subtitles_dir)):
#    print(f'processing {file}')
#
#    lines = []
#    with open(subtitles_dir + file, 'r') as f:
#        lines = f.readlines()
#
#    clips = []
#    i = 0
#    while (i+4 <= len(lines)+1):
#        title = lines[i]
#        start = lines[i+1]
#        end = lines[i+2]
#        d = {}
#        d['title'] = title.strip()
#        d['start'] = start.strip()
#        d['end'] = end.strip()
#        clips.append(d)
#        i += 4
#    
#    print('resetting subtitle section numbers')
#    for i in range(1, len(clips)+1):
#        clips[i-1]['title'] = i
#
#    print('writing to file')
#    with open(subtitles_dir + file, 'w') as f:
#        for clip in clips:
#            f.write(str(clip['title']) + '\n')
#            f.write(clip['start'] + '\n')
#            f.write(clip['end'] + '\n')
#            f.write('\n')
#    print('---')

In [ ]:
def extract_start_stop_from_subtitle_section(timestamp_string):
    timestamps = timestamp_string.split(' ')
    start = timestamps[0]
    end = timestamps[2]
    return start, end

In [ ]:
def get_total_seconds_value_of_timestamp(timestamp):
    hours = int(timestamp[0:2])
    minutes = int(timestamp[3:5])
    seconds = int(timestamp[6:8])
    milliseconds = int(timestamp[9:])
    total_seconds = 60*60*hours + 60*minutes + seconds + milliseconds/1000
    return total_seconds

In [ ]:
def time_ascending_error(section):
    global timestamp_count_seconds
    timestamp_string = section['timestamps']
    start, end = extract_start_stop_from_subtitle_section(timestamp_string)
    start = get_total_seconds_value_of_timestamp(start)
    end = get_total_seconds_value_of_timestamp(end)
    if (start < timestamp_count_seconds): return True
    timestamp_count_seconds = start
    if (end < timestamp_count_seconds): return True
    timestamp_count_seconds = end
    return False


In [ ]:
#verifying subtitle files after correction
timestamp_count_seconds = 0.0
for file in sorted(os.listdir(subtitles_dir)):
    print(f'verifying {file}')

    lines =  []
    with open(subtitles_dir + file, 'r') as f:
        lines = f.readlines()
    
    sections = []
    i = 0
    current_section_num = 0
    while (i+4 <= len(lines)+1):
        section_num = lines[i]
        timestamps = lines[i+1]
        subtitle = lines[i+2]
        d = {}
        d['num'] = section_num.strip()
        d['timestamps'] = timestamps.strip()
        d['subtitle'] = subtitle.strip()
        sections.append(d)
        i += 4

    current_section_num = 0
    timestamp_count_seconds = 0.0
    pattern = r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}'
    for section in sections:
        if int(section['num']) <= current_section_num: print(f'Repeating or lower section number - {section["num"]}')
        if re.fullmatch(pattern, section['timestamps']) == None: print(f'Timestamp pattern error in section {section["num"]}')
        if time_ascending_error(section): print(f'Time Ascending Error in section {section["num"]}')
        current_section_num = int(section['num'])
    
    print('-----')

In [ ]:
#Plot a video frame in loop to figure out the cropping with a rectangle on top
#for title, data in subclip_data.items():
# 
#    #if int(title[0:2]) != 11: continue
#    main_clip = VideoFileClip(original_video_file)
#    main_clip.save_frame(temp_dir + 'crop_base_image' + png, t=convert_str_time_to_seconds(data['start_time']))
# 
#    inp = ''
#    cmd = ''
#    mode = 'square'
#    width = clip_dims[mode]['width']
#    height = clip_dims[mode]['height']
#    crop_x = 0.5*1920 - 0.5*width
#    white_left_x = 0
#    white_right_x = crop_x + width
#    white_left_width = crop_x
#    white_right_width = 1920 - white_left_width - width
#    
#    def clear():
#        plt.close()
#        clear_output()
#
#    def mode_handler():
#        global width 
#        global height 
#        global crop_x
#        global mode
#        args = inp.split(' ')[1]
#        if (args == 'p'): mode = 'portrait'
#        if (args == 's'): mode = 'square'
#        width = clip_dims[mode]['width']
#        height = clip_dims[mode]['height']
#        crop_x = 0.5*1920 - 0.5*width
#        update_white_rectangles()
#
#    def offset_handler():
#        global crop_x
#        args = inp.split(' ')[1:]
#        val = args[0]
#        dir = args[1]
#        if dir == 'l': 
#            crop_x = crop_x - int(val)
#        if dir == 'r': 
#            crop_x = crop_x + int(val)
#        update_white_rectangles()
#        print(f'crop_x - {crop_x}')
#
#    def plot_fn():
#        global width 
#        global height 
#        global x
#        fig, ax = plt.subplots(figsize=(10, 8))
#        im = Image.open(temp_dir + 'crop_base_image' + png)
#        crop_rect = Rectangle((crop_x, 0), width, height, linewidth=1, edgecolor='g', facecolor='none')
#        white_left_rect = Rectangle((white_left_x, 0), white_left_width, height, linewidth=0, facecolor='white', alpha=0.8)
#        white_right_rect = Rectangle((white_right_x, 0), white_right_width, height, linewidth=0, facecolor='white', alpha=0.8)
#        ax.add_patch(crop_rect)
#        ax.add_patch(white_left_rect)
#        ax.add_patch(white_right_rect)
#        ax.imshow(im)
#        plt.show(block=False)
#        plt.pause(0.1)
#        
#    def reset_handler():
#        global mode
#        global crop_x
#        global width
#        global height
#        global white_left_x
#        global white_right_x
#        global white_left_width
#        global white_right_width
#
#        mode = ''
#        crop_x = 0
#        width = 100
#        height = 100
#        white_left_x = 0
#        white_right_x = crop_x + width
#        white_left_width = crop_x
#        white_right_width = 1920 - white_left_width - width
#
#    def set_handler(): 
#        global mode
#        with open(input_json_file_location, 'r') as f:
#            fileData = json.load(f)
#        fileData[title]['aspect_ratios'][mode]['crop_x_start'] = crop_x
#        with open(input_json_file_location, 'w') as f:
#            json.dump(fileData, f, indent=2)
#
#    def update_white_rectangles():
#        global crop_x
#        global white_left_x
#        global white_right_x
#        global white_left_width
#        global white_right_width
#        global width
#        white_left_x = 0
#        white_left_width = crop_x
#        white_right_x = white_left_width + width
#        white_right_width = 1920 - white_left_width - width
#
#    while cmd != 'exit' or cmd != 'next':
#        cmd = inp.split(' ')[0]
#        if (cmd == 'exit'): break
#        elif (cmd == 'm'): mode_handler()
#        elif (cmd == 'mv'): offset_handler()
#        elif (cmd == 'next'): break
#        elif (cmd == 'reset'): reset_handler()
#        elif (cmd == 'set') : set_handler()
#        print(f'processing {title}')
#        print(f'mode - {mode}')
#        print(f'start time {data["start_time"]}')
#        print(f'file crop value - {data["aspect_ratios"][mode]["crop_x_start"]}')
#        plot_fn()
#        inp = input('command: ')
#        clear()
#    if (cmd == 'exit'): break
#

In [ ]:
#input_file.json property setter

#def validate_logo_file_and_position_input(logo_file, position):
#    if int(logo_file) not in [1, 2, 3, 4, 5, 6] or position not in ['l', 'r']: return False
#    else: return True
#
#def get_logo_file_and_position(inp):
#    global last_set_value
#    if (last_set_value != ''):
#        logo_file = last_set_value[0]
#        logo_position = last_set_value[1]
#    else:
#        inp_valid = False
#        while (inp_valid == False):
#            logo_file = inp[0]
#            logo_position = inp[1]
#            inp_valid = validate_logo_file_and_position_input(logo_file, logo_position)
#            if (inp_valid == False): print(f'Incorrect data entered - {inp}')
#    return logo_file, logo_position
#
#def set_logo_file_and_position_in_data(data, aspect_ratio, logo_file, position):
#    global last_set_value
#    data['aspect_ratios'][aspect_ratio]['logo_file'] = logo_file
#    data['aspect_ratios'][aspect_ratio]['logo_position'] = position
#    last_set_value = f'{logo_file}{position}'
#    return data
#
#def write_file_to_disk(writeData):
#    with open(input_json_file_location, 'w') as f:
#        json.dump(writeData, f, indent=2)
#
#def generate_clip_preview(title, data, mode):
#    global last_set_value
#    clip = generate_subclip_without_crop(original_video_file, data['start_time'], data['end_time'])
#    clip = crop_clip(clip, ap, data['aspect_ratios'][mode]['crop_x_start'])
#    logo_num = data['aspect_ratios'][mode]['logo_file'] 
#    logo_pos = data['aspect_ratios'][mode]['logo_position'] 
#    print('Before first if condition')
#    print(logo_num)
#    print(logo_pos)
#    print('--------')
#    if (logo_num != None and logo_pos != None):
#        print('In first if condition')
#        print(logo_num)
#        print(logo_pos)
#        print('--------')
#        logo_pos = 'right' if logo_pos == 'r' else 'left'
#        logo = (ImageClip(logo_symbol_dir + 'symbol' + logo_num + png)
#                .set_duration(clip.duration)
#                .set_position(logo_props[mode][logo_pos]['symbol_position'])
#                .resize(logo_props[mode][logo_pos]['symbol_resize_factor'])
#                .crossfadeout(0.3))
#        handle = (TextClip('SASHA COBRA', font='Montserrat-SemiBold', 
#                           fontsize=logo_props[mode][logo_pos]['text_fontsize'], 
#                           color=logo_colours[int(logo_num)], 
#                           align='center', kerning=5)
#                  .set_position(logo_props[mode][logo_pos]['text_position'])
#                  .set_duration(clip.duration)
#                  .crossfadeout(0.3))
#        clip = CompositeVideoClip([clip, logo, handle]) 
#    if (logo_num == None and logo_pos == None and last_set_value != ''):
#        logo_num = last_set_value[0]
#        logo_pos = last_set_value[1]
#        logo_pos = 'right' if logo_pos == 'r' else 'left'
#        logo = (ImageClip(logo_symbol_dir + 'symbol' + logo_num + png)
#                .set_duration(clip.duration)
#                .set_position(logo_props[mode][logo_pos]['symbol_position'])
#                .resize(logo_props[mode][logo_pos]['symbol_resize_factor'])
#                .crossfadeout(0.3))
#        handle = (TextClip('SASHA COBRA', font='Montserrat-SemiBold', 
#                           fontsize=logo_props[mode][logo_pos]['text_fontsize'], 
#                           color=logo_colours[int(logo_num)], 
#                           align='center', kerning=5)
#                  .set_position(logo_props[mode][logo_pos]['text_position'])
#                  .set_duration(clip.duration)
#                  .crossfadeout(0.3))
#        clip = CompositeVideoClip([clip, logo, handle]) 
#    clip.save_frame(temp_dir + 'logo_test.png')
#    fig, ax = plt.subplots(figsize=(10, 8))
#    im = Image.open(temp_dir + 'logo_test' + png)
#    ax.imshow(im)
#    plt.show(block=False)
#    plt.pause(0.1)
#
##Loading input json file
#with open(input_json_file_location) as f:
#    input_json_data = json.load(f)
#
#last_set_value = ''
#for title, data in input_json_data.items():
#   if int(title[0:2]) != 11: continue
#   for ap in data['aspect_ratios'].keys():
#         while (True):
#            plt.close()
#            clear_output()
#            print(f'Processing {title}') 
#            print(f'mode - {ap}')
#            print(f'current file value - logo_num: {data["aspect_ratios"][ap]["logo_file"]} logo_pos: {data["aspect_ratios"][ap]["logo_position"]}')
#            print(f'last set value - {last_set_value}')
#            if (last_set_value != ''):
#                print('applying last set value')
#                logo_file, logo_postition = get_logo_file_and_position('')
#                set_logo_file_and_position_in_data(data, ap, logo_file, logo_position)
#            print('generating preview ...')
#            generate_clip_preview(title, data, ap)
#            inp = input('Logo file and position or Y to set')
#            if (inp == 'Y'):
#                write_file_to_disk(input_json_data)
#                break
#            elif (inp == 'exit' or inp == 'next'):
#                break
#            else: 
#                last_set_value = ''
#                logo_file, logo_position = get_logo_file_and_position(inp)
#                data = set_logo_file_and_position_in_data(data, ap, logo_file, logo_position)
#         if (inp == 'exit' or inp == 'next'): break
#   if (inp == 'exit'): break

In [ ]:
#validate input_file.json
def validate_input_json():

    with open(input_json_file_location) as f:
        input_json_data = json.load(f)
    
    for title, data in input_json_data.items():
        print(f'validating - {title}')
        if (dot_or_comma_or_digit_error(data['start_time'])): print(f"Dot, comma or digit error in start time - {data['start_time']}")
        if (dot_or_comma_or_digit_error(data['end_time'])): print(f"Dot, comma or digit error in start time - {data['end_time']}")
        for ar, ar_data in data['aspect_ratios'].items():
            if (ar_data['logo_file'] == None): print(f'Logo file for {ar} is None')
            if (ar_data['logo_position'] == None): print(f'Logo file for {ar} is None')
        print('----')

validate_input_json()

In [ ]:
#Final Generate Dump

with open(input_json_file_location) as f:
    input_json_data = json.load(f)

for title, data in input_json_data.items():
    
    #if (int(title[0:2]) < 23): continue
    print(f'processing {title}')

    for ar, ar_data in data['aspect_ratios'].items():
        #if (ar != 'landscape'): continue
        clip = generate_subclip_without_crop(original_video_file, data['start_time'], data['end_time'])
        clip = crop_clip(clip, ar, ar_data['crop_x_start'])
        logo_num = ar_data['logo_file'] 
        logo_pos = ar_data['logo_position']
        logo_pos = 'right' if logo_pos == 'r' else 'left'
        logo = (ImageClip(logo_symbol_dir + 'symbol' + logo_num + png)
                .set_duration(clip.duration)
                .set_position(logo_props[ar][logo_pos]['symbol_position'])
                .resize(logo_props[ar][logo_pos]['symbol_resize_factor'])
                .crossfadeout(0.3))
        handle = (TextClip('SASHA COBRA', font='Montserrat-SemiBold', 
                           fontsize=logo_props[ar][logo_pos]['text_fontsize'], 
                           color=logo_colours[int(logo_num)], 
                           align='center', kerning=5)
                  .set_position(logo_props[ar][logo_pos]['text_position'])
                  .set_duration(clip.duration)
                  .crossfadeout(0.3))
        clip = CompositeVideoClip([clip, logo, handle]) 

        min, sec = divmod(clip.duration, 60)
        min = int(min)
        sec = int(sec)
        save_path = root_dir + current_video_dir + f'{ar}_logo/{title}_{min}m_{sec}s{mp4}'
        clip.write_videofile(save_path, audio_codec='aac')

        clear_output(wait=True)

        #generate clips with subtitles
        subtitle_file = subtitles_dir + title + '.srt'

        #Extract the data in subtiitle file
        subtitle = subtitle_file_to_dict(subtitle_file)
    
        #Stitching the subtitles
        clip = stitch_subtitles(subtitle, clip, ar)

        save_path = root_dir + current_video_dir + f'{ar}_logo_subtitles/{title}_{min}m_{sec}s{mp4}'
        clip.write_videofile(save_path, audio_codec='aac')

        clear_output(wait=True)

        post_clip_anim_input_params = {
            'end_time': hmsms_to_tuple(data['end_time']),
            'title': title,
            'mode': ar,
            'crop_x_start': data['aspect_ratios'][ar]['crop_x_start']
        }
        post_clip_anim = add_post_clip_ani(post_clip_anim_input_params)
        clip = concatenate_videoclips([clip, post_clip_anim])
        min, sec = divmod(clip.duration, 60)
        min = int(min)
        sec = int(sec)
        save_path = root_dir + current_video_dir + f'{ar}_logo_subtitles_outro/{title}_{min}m_{sec}s{mp4}'
        clip.write_videofile(save_path, audio_codec='aac')

        clear_output(wait=True)

In [ ]:
#function to generate a large video clip by splitting it into smaller clips
#def large_clip_subtitle_generator(title):
#
#    #Load subtitle file as dict
#    subtitle_file = subtitles_dir + title + '.srt'
#    subtitle = subtitle_file_to_dict(subtitle_file)
#    last_subtitle_index = list(subtitle.keys())[-1]
#
#    i = 0
#    clips = []
#    transition_clips = []
#    previous_end_time = None
#    while (i <= len(subtitle.keys())):
#        print(f'Generating Clip {i+1} subtitles')
#        start_time = subtitle[i+1]["start"];
#        if (i + 1 + 30 > last_subtitle_index): end_time = subtitle[last_subtitle_index]["start"]
#        else: end_time = subtitle[i + 1 + 30]["start"]
#        print(f'start time - {start_time}')
#        print(f'end time - {end_time}')
#        range_end = last_subtitle_index if (i+1+30) > last_subtitle_index else (i+1+30)
#        clip_subtitle = dict((k, subtitle[k]) for k in range(i+1, range_end))
#        clip_subtitle = dict((k, clip_subtitle[k + i]) for k in range(1, len(clip_subtitle.keys())))
#        clips.append(clip_subtitle)
#        print(clip_subtitle)
#        print('--------------------')
#        i += 30
#    
#    for i in range(len(clips)):
#        
#
#    #with open(long_video_clip_processor_dir + 'test.txt', 'w+') as f:
#    #    f.write('Hey')
#
#large_clip_subtitle_generator('16_parenting_without_limits')

In [ ]:
#Code that could be used later

#            #temp_save_path = temp_dir + f'test_sub_palce_port.png'
#            #clip.save_frame(temp_save_path, t=5)
#            #fig, ax = plt.subplots(figsize=(10, 8))
#            #im = Image.open(temp_dir + 'test_sub_palce_port' + png)
#            #ax.imshow(im)
#            #plt.show(block=False)
#            #plt.pause(0.1)

In [ ]:
#looping through all subclips
#generating 20 frames from each subclip and checking if 
#camera position changes or background changes
#for title, data in subclip_data.items():
#    print(f'processing {title}')
#    subclip = generate_subclip_without_crop(original_video_file, data['start_time'], data['end_time'])
#    frame_timestamps = generate_random_unique_timestamps(subclip)
#    save_random_frames(subclip, frame_timestamps)
#    make_and_save_grid_of_random_frames(title)
#    delete_randomly_generate_frames()
#    clear_output(wait=True)


In [ ]:
#delete logo file preview checks
#def delete_logo_file_previews():
#    for i in range(1, 7):
#        file_path = temp_dir + f'logo_test_{i}.png'
#        os.remove(file_path)

In [ ]:
#generate random frame timestamps
#def generate_random_unique_timestamps(subclip):
#    frame_timestamps = []
#    for i in range(num_frame_checks):
#        random_num = random.uniform(0, int(subclip.duration))
#        if random_num not in frame_timestamps: frame_timestamps.append(random_num)
#    frame_timestamps.sort()
#    return frame_timestamps

In [ ]:
#make and save grid of random frame images
#def make_and_save_grid_of_random_frames(title):
#
#    print('generating grid for ' + title)
#
#    image_paths = glob.glob(temp_dir + '*.png')
#
#    image_arr = []
#    for path in image_paths:
#        image_arr.append(img_resize(path, 10))
#
#    fig = plt.figure(figsize=(384, 216))
#    grid = ImageGrid(
#        fig,
#        111,
#        nrows_ncols = (2, 10),
#        axes_pad = 0.1
#    )
#
#    for ax, im in zip(grid, image_arr):
#        ax.imshow(im)
#
#    save_path = random_frames_camera_check_dir + title + png
#    plt.savefig(save_path)

In [ ]:
#delete randomly saved frames in temp_dir
#def delete_randomly_generate_frames():
#    for i in range(num_frame_checks):
#        file_path = temp_dir + 'frame_' + str(i) + png
#        os.remove(file_path)

In [ ]:
#given list of random timestamps in clip, saves frames at timestamps in png format
#def save_random_frames(subclip, frame_timestamps):
#    for i in range(num_frame_checks): 
#        save_path = temp_dir + 'frame_' + str(i) + png
#        print('saving ' + save_path)
#        subclip.save_frame(save_path, t=frame_timestamps[i])